# Лабораторная работа №2 (Проведение исследований с логистической и линейной регрессией)


In [10]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    mean_squared_error, mean_absolute_error, r2_score
)

## 2. Создание бейзлайна и оценка качества

**2.a** Обучить модели из sklearn (для классификации и регрессии) для выбранных наборов данных  
**2.b** Оценить качество моделей (для классификации и регрессии) по выбранным метрикам на выбранных наборах данных

In [8]:
# 1. Загрузка данных
data = pd.read_csv('student_data.csv', sep=',')

data_class = data.copy()
data_class['passed'] = (data_class['G3'] >= 10).astype(int)
X_class = data_class[['G1', 'G2', 'studytime', 'failures', 'absences']]
y_class = data_class['passed']

# Разделим на train/test
Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# 3. Бейзлайн для классификации (Logistic Regression)
logreg_baseline = LogisticRegression(max_iter=200, random_state=42)
logreg_baseline.fit(Xc_train, yc_train)
yc_pred_baseline = logreg_baseline.predict(Xc_test)

acc = accuracy_score(yc_test, yc_pred_baseline)
prec = precision_score(yc_test, yc_pred_baseline)
rec = recall_score(yc_test, yc_pred_baseline)
f1 = f1_score(yc_test, yc_pred_baseline)

print("Бейзлайн (Logistic Regression) - Classification:")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")

# 4. Регрессия: Предсказываем G3
X_reg = data[['G1', 'G2', 'studytime', 'failures', 'absences']]
y_reg = data['G3']

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Бейзлайн для регрессии (Linear Regression)
linreg_baseline = LinearRegression()
linreg_baseline.fit(Xr_train, yr_train)
yr_pred_baseline = linreg_baseline.predict(Xr_test)

mse = mean_squared_error(yr_test, yr_pred_baseline)
mae = mean_absolute_error(yr_test, yr_pred_baseline)
r2 = r2_score(yr_test, yr_pred_baseline)

print("\nБейзлайн (Linear Regression) - Regression:")
print(f"MSE:  {mse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"R^2:  {r2:.4f}")

Бейзлайн (Logistic Regression) - Classification:
Accuracy:  0.8734
Precision: 0.9388
Recall:    0.8679
F1-score:  0.9020

Бейзлайн (Linear Regression) - Regression:
MSE:  4.4665
MAE:  1.3394
R^2:  0.7822


**Вывод**:  
- Логистическая регрессия даёт определённые метрики (accuracy, precision, recall, f1) для задачи "сдал/не сдал".  
- Линейная регрессия даёт базовые результаты MSE, MAE, R^2 для предсказания итоговой оценки G3.


# 3. Улучшение бейзлайна
**3.a** Сформулировать гипотезы (предобработка данных, визуализация, новые признаки, подбор гиперпараметров и т.д.)  
**3.b** Проверить гипотезы  
**3.c** Сформировать улучшенный бейзлайн  
**3.d** Обучить модели с улучшенным бейзлайном (логистическая и линейная регрессия)  
**3.e** Оценить качество моделей с улучшенным бейзлайном  
**3.f** Сравнить результаты моделей с улучшенным бейзлайном с пунктом 2  
**3.g** Сделать выводы

In [11]:
# ====================================
# ЧАСТЬ КОДА УЛУЧШЕНИЯ БЕЙЗЛАЙНА
# ====================================

# Удалим пропуски (если они есть; если нет - этот шаг пропустится)
data.dropna(inplace=True)

# Пример удаления выбросов по признаку 'absences' (здесь порог 40 как пример)
data = data[data['absences'] <= 40]

# Создадим новый признак total_G1_G2
data['total_G1_G2'] = data['G1'] + data['G2']

# ----- (2) УЛУЧШЕНИЕ ДЛЯ КЛАССИФИКАЦИИ: ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ -----

# Целевая переменная (сдал / не сдал)
data_class = data.copy()
data_class['passed'] = (data_class['G3'] >= 10).astype(int)
X_class = data_class[['G1', 'G2', 'studytime', 'failures', 'absences', 'total_G1_G2']]
y_class = data_class['passed']

# Разделяем
Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Пайплайн для улучшенной модели: масштабирование + лог. регрессия
pipe_logreg = Pipeline([
    ('scaler', StandardScaler()),  # Можно менять на MinMaxScaler, RobustScaler
    ('logreg', LogisticRegression(solver='saga', max_iter=1000, random_state=42))
])

# Сетка гиперпараметров
param_grid_logreg = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler()],
    'logreg__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'logreg__penalty': ['l1','l2']
}

# GridSearch
grid_logreg = GridSearchCV(pipe_logreg, param_grid_logreg, cv=5, scoring='f1', n_jobs=-1)
grid_logreg.fit(Xc_train, yc_train)

best_logreg = grid_logreg.best_estimator_
yc_pred = best_logreg.predict(Xc_test)

acc_best = accuracy_score(yc_test, yc_pred)
prec_best = precision_score(yc_test, yc_pred)
rec_best = recall_score(yc_test, yc_pred)
f1_best = f1_score(yc_test, yc_pred)

print("===== Улучшенный бейзлайн (Логист. регрессия) =====")
print("Лучшие параметры:", grid_logreg.best_params_)
print(f"Accuracy:  {acc_best:.4f}")
print(f"Precision: {prec_best:.4f}")
print(f"Recall:    {rec_best:.4f}")
print(f"F1-score:  {f1_best:.4f}")

# ----- (3) УЛУЧШЕНИЕ ДЛЯ РЕГРЕССИИ: RIDGE + ПОЛИНОМИАЛЬНЫЕ ПРИЗНАКИ -----

# Целевая переменная: G3
X_reg = data[['G1', 'G2', 'studytime', 'failures', 'absences', 'total_G1_G2']]
y_reg = data['G3']

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Пайплайн: масштабируем, делаем полиномиальные признаки, потом Ridge
pipe_ridge = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(include_bias=False)),
    ('ridge', Ridge(random_state=42))
])

param_grid_ridge = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler()],
    'poly__degree': [1, 2],  # пробуем линейные и квадратичные признаки
    'ridge__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

grid_ridge = GridSearchCV(
    pipe_ridge, param_grid_ridge,
    cv=5, scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_ridge.fit(Xr_train, yr_train)
best_ridge = grid_ridge.best_estimator_
yr_pred = best_ridge.predict(Xr_test)

mse_best = mean_squared_error(yr_test, yr_pred)
mae_best = mean_absolute_error(yr_test, yr_pred)
r2_best = r2_score(yr_test, yr_pred)

print("\n===== Улучшенный бейзлайн (Ridge регрессия) =====")
print("Лучшие параметры:", grid_ridge.best_params_)
print(f"MSE:  {mse_best:.4f}")
print(f"MAE:  {mae_best:.4f}")
print(f"R^2:  {r2_best:.4f}")

===== Улучшенный бейзлайн (Логист. регрессия) =====
Лучшие параметры: {'logreg__C': 1, 'logreg__penalty': 'l1', 'scaler': MinMaxScaler()}
Accuracy:  0.8608
Precision: 0.9038
Recall:    0.8868
F1-score:  0.8952

===== Улучшенный бейзлайн (Ridge регрессия) =====
Лучшие параметры: {'poly__degree': 2, 'ridge__alpha': 0.1, 'scaler': MinMaxScaler()}
MSE:  2.4281
MAE:  1.1251
R^2:  0.8264


**Вывод**:
- Мы проверили гипотезу о том, что при логистической регрессии поможет масштабирование и подбор `C`, `penalty`, `solver`.  
- Для линейной регрессии (заменили на Ridge) попробовали масштабирование, полиномиальные признаки и гиперпараметр `alpha`.  
- Сравните полученные метрики с бейзлайном из пункта 2.


# 4. Имплементация алгоритма машинного обучения
**4.a** Самостоятельно имплементировать алгоритмы (логистическую и линейную регрессию)  
**4.b** Обучить имплементированные модели (логистическая и линейная)  
**4.c** Оценить качество имплементированных моделей  
**4.d** Сравнить результаты имплементированных моделей с результатами из пункта 2  
**4.e** Сделать выводы  
**4.f** Добавить техники из улучшенного бейзлайна (пункт 3.c)  
**4.g** Обучить модели (логистическую и линейную)  
**4.h** Оценить качество этих моделей  
**4.i** Сравнить результаты моделей с пунктом 3  
**4.j** Сделать выводы

In [20]:


####################
# 4.1 Имплементация ЛИНЕЙНОЙ РЕГРЕССИИ
####################
class CustomLinearRegression:
    def __init__(self, lr=0.01, n_iter=1000):
        self.lr = lr
        self.n_iter = n_iter
    
    def fit(self, X, y):
        X = np.array(X, dtype=float)
        y = np.array(y, dtype=float)
        
        # Добавляем столбец 1 (bias)
        ones = np.ones((X.shape[0], 1))
        X = np.hstack([ones, X])
        
        # Инициализация весов
        self.w_ = np.zeros(X.shape[1])
        
        # Градиентный спуск
        for _ in range(self.n_iter):
            y_pred = X.dot(self.w_)
            error = y_pred - y
            grad = (1 / X.shape[0]) * X.T.dot(error)
            self.w_ -= self.lr * grad
        return self
    
    def predict(self, X):
        X = np.array(X, dtype=float)
        ones = np.ones((X.shape[0], 1))
        X = np.hstack([ones, X])
        return X.dot(self.w_)

####################
# 4.2 Имплементация ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ
####################
class CustomLogisticRegression:
    def __init__(self, lr=0.0001, n_iter=1000):
        self.lr = lr
        self.n_iter = n_iter
    
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def fit(self, X, y):
        X = np.array(X, dtype=float)
        y = np.array(y, dtype=float)
        
        # Добавляем столбец 1 (bias)
        ones = np.ones((X.shape[0], 1))
        X = np.hstack([ones, X])
        
        self.w_ = np.zeros(X.shape[1])
        
        for _ in range(self.n_iter):
            z = X.dot(self.w_)
            y_pred = self._sigmoid(z)
            error = y_pred - y
            grad = (1 / X.shape[0]) * X.T.dot(error)
            self.w_ -= self.lr * grad
        return self
    
    def predict_proba(self, X):
        X = np.array(X, dtype=float)
        ones = np.ones((X.shape[0], 1))
        X = np.hstack([ones, X])
        return self._sigmoid(X.dot(self.w_))
    
    def predict(self, X, threshold=0.5):
        proba = self.predict_proba(X)
        return (proba >= threshold).astype(int)

# ------------------------
# ШАГ 4. ОБУЧАЕМ КАСТОМНЫЕ МОДЕЛИ (без масштабирования)
# ------------------------

# 4.a Логистическая регрессия (Classification)
custom_logreg = CustomLogisticRegression(lr=0.01, n_iter=2000)
custom_logreg.fit(Xc_train, yc_train)
yc_pred_custom = custom_logreg.predict(Xc_test)

acc_custom = accuracy_score(yc_test, yc_pred_custom)
prec_custom = precision_score(yc_test, yc_pred_custom)
rec_custom = recall_score(yc_test, yc_pred_custom)
f1_custom = f1_score(yc_test, yc_pred_custom)

print("\n=== Custom Logistic Regression - Classification (без масштабирования) ===")
print(f"Accuracy:  {acc_custom:.4f}")
print(f"Precision: {prec_custom:.4f}")
print(f"Recall:    {rec_custom:.4f}")
print(f"F1-score:  {f1_custom:.4f}")

# 4.b Линейная регрессия (Regression)
custom_linreg = CustomLinearRegression(lr=0.001, n_iter=2000)
custom_linreg.fit(Xr_train, yr_train)
yr_pred_custom = custom_linreg.predict(Xr_test)

mse_custom = mean_squared_error(yr_test, yr_pred_custom)
mae_custom = mean_absolute_error(yr_test, yr_pred_custom)
r2_custom = r2_score(yr_test, yr_pred_custom)

print("\n=== Custom Linear Regression - Regression (без масштабирования) ===")
print(f"MSE:  {mse_custom:.4f}")
print(f"MAE:  {mae_custom:.4f}")
print(f"R^2:  {r2_custom:.4f}")

# ------------------------
# ШАГ 5. УЛУЧШЕНИЕ БЕЙЗЛАЙНА (МАСШТАБИРОВАНИЕ)
# ------------------------
scaler_class = StandardScaler()
Xc_train_scaled = scaler_class.fit_transform(Xc_train)
Xc_test_scaled = scaler_class.transform(Xc_test)

scaler_reg = StandardScaler()
Xr_train_scaled = scaler_reg.fit_transform(Xr_train)
Xr_test_scaled = scaler_reg.transform(Xr_test)

# 5.a Логистическая регрессия (scaled)
custom_logreg_scaled = CustomLogisticRegression(lr=0.01, n_iter=2000)
custom_logreg_scaled.fit(Xc_train_scaled, yc_train)
yc_pred_custom_scaled = custom_logreg_scaled.predict(Xc_test_scaled)

acc_custom_s = accuracy_score(yc_test, yc_pred_custom_scaled)
f1_custom_s = f1_score(yc_test, yc_pred_custom_scaled)

print("\n===Масштабирование===")
print("\n=== Custom Logistic Regression - Classification (scaled) ===")
print(f"Accuracy:  {acc_custom_s:.4f}")
print(f"F1-score:  {f1_custom_s:.4f}")

# 5.b Линейная регрессия (scaled)
custom_linreg_scaled = CustomLinearRegression(lr=0.01, n_iter=2000)
custom_linreg_scaled.fit(Xr_train_scaled, yr_train)
yr_pred_custom_scaled = custom_linreg_scaled.predict(Xr_test_scaled)

mse_custom_s = mean_squared_error(yr_test, yr_pred_custom_scaled)
r2_custom_s = r2_score(yr_test, yr_pred_custom_scaled)

print("\n=== Custom Linear Regression - Regression (scaled) ===")
print(f"MSE:  {mse_custom_s:.4f}")
print(f"R^2:  {r2_custom_s:.4f}")



=== Custom Logistic Regression - Classification (без масштабирования) ===
Accuracy:  0.8228
Precision: 0.8421
Recall:    0.9057
F1-score:  0.8727

=== Custom Linear Regression - Regression (без масштабирования) ===
MSE:  4.2771
MAE:  1.2219
R^2:  0.7914

===Масштабирование===

=== Custom Logistic Regression - Classification (scaled) ===
Accuracy:  0.8354
F1-score:  0.8713

=== Custom Linear Regression - Regression (scaled) ===
MSE:  4.4633
R^2:  0.7823
